In [6]:
# The idea is to train a model for each of the aspect
# And from each model get the prob of ai and human
# And make a final prediction by summing the prob of all of them

In [1]:
import json
from tf_idf import *
from NN_train import *

In [2]:
vocab_path = "/Data/sandeep/Vardhan/Journal/vocabulary.json"
vocab = json.load(open(vocab_path))

In [3]:
def get_data(folders,path):
    data={}
    for f in folders:
        p = os.path.join(path, f)   
        for file in os.listdir(p):
            try:
                revw = json.load(open(os.path.join(p,file)))
                data[list(revw.keys())[0]] = list(revw.values())[0]
            except:
                pass
                # print(f"Error at {file} in folder {f}")

    return data

In [4]:
def ultra_get_data(path):
    folders = ["ai_iclr","ai_neur","ai_org","human_iclr","human_neur","human_org"]

    ai_iclr = process_aspects(get_data([folders[0]], path))
    ai_neur = process_aspects(get_data([folders[1]], path))
    ai_org = process_aspects(get_data([folders[2]], path))
    human_iclr = process_aspects(get_data([folders[3]], path))
    human_neur = process_aspects(get_data([folders[4]], path))
    human_org = process_aspects(get_data([folders[5]], path))

    # We just have work of reviews
    data = {
        "iclr":{"ai":{id:ai_iclr[id]['review'] for id in list(ai_iclr.keys())},
        "human":{id:human_iclr[id]['review'] for id in list(human_iclr.keys())}},
        "neur":{"ai":{id:ai_neur[id]['review'] for id in list(ai_neur.keys())},
        "human":{id:human_neur[id]['review'] for id in list(human_neur.keys())}},
        "org":{"ai":{id:ai_org[id]['review'] for id in list(ai_org.keys())},
        "human":{id:human_org[id]['review'] for id in list(human_org.keys())}},
    }

    return data

In [5]:
path = "/Data/sandeep/Vardhan/Journal/Dataset/Aspects"
data = ultra_get_data(path)

In [6]:
ai_dict = json.load(open("ai_dict.json"))
human_dict = json.load(open("human_dict.json"))

## Getting aspect wise data 

In [7]:
aspects = ['motivation', 'originality', 'soundness', 'clarity']

In [8]:
# probability_finder(data['iclr']['ai']['fM8VzFD_2-'], ai_dict['clarity'])

In [9]:
best_params = {'num_hidden_layers': 2,
 'neurons_per_layer': 32,
 'epochs': 150,
 'learning_rate': 0.055825772063256424,
 'dropout_rate': 0.2,
 'batch_size': 16,
 'optimizer_name': 'RMSprop',
 'weight_decay': 0.0003307982403899191,
 "input_dim":2,
 "output_dim":1}


## Training Models

In [ ]:
# I have to train 4 models, on 3 different domains 
# So there will be total 3 test sets

In [10]:
def model_trainer(ai,human, model_params):
    input_dim = model_params['input_dim']
    output_dim = model_params['output_dim']
    batch_size = model_params['batch_size']
    epochs = model_params['epochs']
    
    X = {id:ai[id]+human[id] for id in list(ai.keys())}
    y = {id:[[1] for _ in range(len(ai[id]))] + [[0] for _ in range(len(human[id]))] for id in list(ai.keys())}

    data = {id:train_test_split(X[id], y[id], test_size=0.2) for id in list(X.keys())}
    X_train = []
    y_train = []
    for id in list(data.keys()):
        X_train += data[id][0]
        y_train += data[id][2]

    test_data = {id:{'X':data[id][1], 'y':data[id][3]} for id in list(data.keys())}

    X_train = torch.tensor(X_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.float32)

    train_dataset = CustomDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)

    model, criterion, optimizer= NN_initializer1(
        **model_params,
    )

    # training loop
    for epoch in tqdm(range(epochs)):
        for batch_features, batch_labels in train_loader:
            batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

            outputs = model(batch_features)
            loss = criterion(outputs, batch_labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    # Testing process
    for id in list(test_data.keys()):
       print(f"{'+'*10} For Domain {id} {'+'*10}")
       X_test = torch.tensor(test_data[id]['X'], dtype=torch.float32)
       y_test = torch.tensor(test_data[id]['y'], dtype=torch.float32)
       test_dataset = CustomDataset(X_test, y_test)
       tester(model,test_dataset)

    return model

In [11]:
model_save_path = "/Data/sandeep/Vardhan/Journal/models"

### Clarity

In [93]:
ASP = aspects[3]
ai = {
    domain:[[probability_finder(revw, ai_dict[ASP]),probability_finder(revw, human_dict[ASP])] for revw in list(data[domain]['ai'].values())] for domain in list(data.keys())
}

human = {
    domain:[[probability_finder(revw, ai_dict[ASP]),probability_finder(revw, human_dict[ASP])] for revw in list(data[domain]['human'].values())] for domain in list(data.keys())
}

In [95]:
clarity_models = []
for i in range(5):
    print(f"{i}")
    clarity_models.append(model_trainer(ai, human, best_params))
    print("\n"*3)

0


100%|██████████| 150/150 [00:58<00:00,  2.58it/s]


++++++++++ For Domain iclr ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0
++++++++++ For Domain neur ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0
++++++++++ For Domain org ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0




1


100%|██████████| 150/150 [01:00<00:00,  2.47it/s]


++++++++++ For Domain iclr ++++++++++
Accuracy : 0.9966555183946488
Precision : 0.9934640522875817
Recall : 1.0
F1-score : 0.9967213114754099
++++++++++ For Domain neur ++++++++++
Accuracy : 0.9930313588850174
Precision : 0.9875776397515528
Recall : 1.0
F1-score : 0.99375
++++++++++ For Domain org ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0




2


...

++++++++++ For Domain iclr ++++++++++
Accuracy : 0.9966555183946488
Precision : 0.9936305732484076
Recall : 1.0
F1-score : 0.9968051118210862
++++++++++ For Domain neur ++++++++++
Accuracy : 0.9860627177700348
Precision : 0.9727891156462585
Recall : 1.0
F1-score : 0.9862068965517241
++++++++++ For Domain org ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0




3


100%|██████████| 150/150 [00:58<00:00,  2.56it/s]


++++++++++ For Domain iclr ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0
++++++++++ For Domain neur ++++++++++
Accuracy : 0.9930313588850174
Precision : 1.0
Recall : 0.9863013698630136
F1-score : 0.993103448275862
++++++++++ For Domain org ++++++++++
Accuracy : 0.9625
Precision : 1.0
Recall : 0.9117647058823529
F1-score : 0.9538461538461539




4


100%|██████████| 150/150 [00:59<00:00,  2.52it/s]

++++++++++ For Domain iclr ++++++++++
Accuracy : 0.9966555183946488
Precision : 1.0
Recall : 0.9932432432432432
F1-score : 0.9966101694915255
++++++++++ For Domain neur ++++++++++
Accuracy : 0.9965156794425087
Precision : 1.0
Recall : 0.9935483870967742
F1-score : 0.9967637540453075
++++++++++ For Domain org ++++++++++
Accuracy : 0.9875
Precision : 1.0
Recall : 0.9777777777777777
F1-score : 0.9887640449438202






In [99]:
torch.save(clarity_models[0], f"{model_save_path}/clarity.pt")

In [100]:
clarity = clarity_models[0]

In [99]:
yup = torch.load(f"{model_save_path}/clarity.pt")

### Motivation

In [109]:
aspects

['motivation', 'originality', 'soundness', 'clarity']

In [110]:
best_params = {'num_hidden_layers': 3,
 'neurons_per_layer': 32,
 'epochs': 150,
 'learning_rate': 0.055825772063256424,
 'dropout_rate': 0.2,
 'batch_size': 16,
 'optimizer_name': 'RMSprop',
 'weight_decay': 0.0003307982403899191,
 "input_dim":2,
 "output_dim":1}


In [111]:
ASP = aspects[0]
ai = {
    domain:[[probability_finder(revw, ai_dict[ASP]),probability_finder(revw, human_dict[ASP])] for revw in list(data[domain]['ai'].values())] for domain in list(data.keys())
}

human = {
    domain:[[probability_finder(revw, ai_dict[ASP]),probability_finder(revw, human_dict[ASP])] for revw in list(data[domain]['human'].values())] for domain in list(data.keys())
}

motivation_models = []
for i in range(5):
    print(f"{i}")
    motivation_models.append(model_trainer(ai, human, best_params))
    print("\n"*3)
# torch.save(model, f"{model_save_path}/motivation.pt")

0


100%|██████████| 150/150 [01:15<00:00,  1.99it/s]


++++++++++ For Domain iclr ++++++++++
Accuracy : 0.9966555183946488
Precision : 1.0
Recall : 0.9932432432432432
F1-score : 0.9966101694915255
++++++++++ For Domain neur ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0
++++++++++ For Domain org ++++++++++
Accuracy : 0.9625
Precision : 1.0
Recall : 0.9387755102040817
F1-score : 0.968421052631579




1


100%|██████████| 150/150 [01:15<00:00,  1.99it/s]


++++++++++ For Domain iclr ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0
++++++++++ For Domain neur ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0
++++++++++ For Domain org ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0




2


...

++++++++++ For Domain iclr ++++++++++
Accuracy : 0.9966555183946488
Precision : 1.0
Recall : 0.9933333333333333
F1-score : 0.9966555183946488
++++++++++ For Domain neur ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0
++++++++++ For Domain org ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0




3


100%|██████████| 150/150 [01:15<00:00,  1.99it/s]


++++++++++ For Domain iclr ++++++++++
Accuracy : 0.9899665551839465
Precision : 0.9870967741935484
Recall : 0.9935064935064936
F1-score : 0.9902912621359223
++++++++++ For Domain neur ++++++++++
Accuracy : 0.9825783972125436
Precision : 0.9695121951219512
Recall : 1.0
F1-score : 0.9845201238390093
++++++++++ For Domain org ++++++++++
Accuracy : 0.9875
Precision : 0.972972972972973
Recall : 1.0
F1-score : 0.9863013698630136




4


100%|██████████| 150/150 [01:15<00:00,  1.99it/s]

++++++++++ For Domain iclr ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0
++++++++++ For Domain neur ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0
++++++++++ For Domain org ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0






In [117]:
motivation = motivation_models[-1]
torch.save(motivation, f"{model_save_path}/motivation.pt")

In [112]:
# ASP = aspects[0]
# ai = {
#     domain:[[probability_finder(revw, ai_dict[ASP]),probability_finder(revw, human_dict[ASP])] for revw in list(data[domain]['ai'].values())] for domain in list(data.keys())
# }

# human = {
#     domain:[[probability_finder(revw, ai_dict[ASP]),probability_finder(revw, human_dict[ASP])] for revw in list(data[domain]['human'].values())] for domain in list(data.keys())
# }

# motivation = model_trainer(ai, human, best_params)
# torch.save(model, f"{model_save_path}/motivation.pt")

### Originality

In [113]:
best_params = {'num_hidden_layers': 3,
 'neurons_per_layer': 32,
 'epochs': 300,
 'learning_rate': 0.055825772063256424,
 'dropout_rate': 0.2,
 'batch_size': 16,
 'optimizer_name': 'RMSprop',
 'weight_decay': 0.0003307982403899191,
 "input_dim":2,
 "output_dim":1}


In [114]:
ASP = aspects[1]
ai = {
    domain:[[probability_finder(revw, ai_dict[ASP]),probability_finder(revw, human_dict[ASP])] for revw in list(data[domain]['ai'].values())] for domain in list(data.keys())
}

human = {
    domain:[[probability_finder(revw, ai_dict[ASP]),probability_finder(revw, human_dict[ASP])] for revw in list(data[domain]['human'].values())] for domain in list(data.keys())
}

originality_models = []

for i in range(5):
    print(f"{i}")
    originality_models.append(model_trainer(ai, human, best_params))
    print("\n"*3)
# torch.save(model, f"{model_save_path}/originality.pt")

0


100%|██████████| 300/300 [02:30<00:00,  1.99it/s]


++++++++++ For Domain iclr ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0
++++++++++ For Domain neur ++++++++++
Accuracy : 0.9651567944250871
Precision : 1.0
Recall : 0.9295774647887324
F1-score : 0.9635036496350365
++++++++++ For Domain org ++++++++++
Accuracy : 0.9625
Precision : 1.0
Recall : 0.9302325581395349
F1-score : 0.963855421686747




1


100%|██████████| 300/300 [02:31<00:00,  1.99it/s]


++++++++++ For Domain iclr ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0
++++++++++ For Domain neur ++++++++++
Accuracy : 0.9930313588850174
Precision : 1.0
Recall : 0.9862068965517241
F1-score : 0.9930555555555556
++++++++++ For Domain org ++++++++++
Accuracy : 0.95
Precision : 1.0
Recall : 0.9090909090909091
F1-score : 0.9523809523809523




2


...

++++++++++ For Domain iclr ++++++++++
Accuracy : 0.9966555183946488
Precision : 1.0
Recall : 0.9937106918238994
F1-score : 0.9968454258675079
++++++++++ For Domain neur ++++++++++
Accuracy : 0.975609756097561
Precision : 1.0
Recall : 0.951048951048951
F1-score : 0.974910394265233
++++++++++ For Domain org ++++++++++
Accuracy : 0.95
Precision : 1.0
Recall : 0.9
F1-score : 0.9473684210526315




3


100%|██████████| 300/300 [02:29<00:00,  2.01it/s]


++++++++++ For Domain iclr ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0
++++++++++ For Domain neur ++++++++++
Accuracy : 0.9895470383275261
Precision : 1.0
Recall : 0.9765625
F1-score : 0.9881422924901185
++++++++++ For Domain org ++++++++++
Accuracy : 0.875
Precision : 1.0
Recall : 0.7560975609756098
F1-score : 0.8611111111111112




4


100%|██████████| 300/300 [02:30<00:00,  1.99it/s]

++++++++++ For Domain iclr ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0
++++++++++ For Domain neur ++++++++++
Accuracy : 0.9721254355400697
Precision : 1.0
Recall : 0.9436619718309859
F1-score : 0.9710144927536232
++++++++++ For Domain org ++++++++++
Accuracy : 0.95
Precision : 1.0
Recall : 0.9024390243902439
F1-score : 0.9487179487179487






In [ ]:
# originality = originality_models[1]
# torch.save(originality, f"{model_save_path}/originality.pt")

### Soundness:

In [115]:
best_params = {'num_hidden_layers': 2,
 'neurons_per_layer': 32,
 'epochs': 200,
 'learning_rate': 0.055825772063256424,
 'dropout_rate': 0.2,
 'batch_size': 16,
 'optimizer_name': 'RMSprop',
 'weight_decay': 0.0003307982403899191,
 "input_dim":2,
 "output_dim":1}


In [ ]:
ASP = aspects[2]
ai = {
    domain:[[probability_finder(revw, ai_dict[ASP]),probability_finder(revw, human_dict[ASP])] for revw in list(data[domain]['ai'].values())] for domain in list(data.keys())
}

human = {
    domain:[[probability_finder(revw, ai_dict[ASP]),probability_finder(revw, human_dict[ASP])] for revw in list(data[domain]['human'].values())] for domain in list(data.keys())
}

soundness_models = []
for i in range(3):
    print(f"{i}")
    soundness_models.append(model_trainer(ai, human, best_params))
    print("\n"*3)
# torch.save(model, f"{model_save_path}/soundness.pt")

0


100%|██████████| 200/200 [01:18<00:00,  2.56it/s]


++++++++++ For Domain iclr ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0
++++++++++ For Domain neur ++++++++++
Accuracy : 0.9965156794425087
Precision : 1.0
Recall : 0.993421052631579
F1-score : 0.9966996699669967
++++++++++ For Domain org ++++++++++
Accuracy : 0.975
Precision : 1.0
Recall : 0.9444444444444444
F1-score : 0.9714285714285714




1


100%|██████████| 200/200 [01:19<00:00,  2.50it/s]


++++++++++ For Domain iclr ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0
++++++++++ For Domain neur ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0
++++++++++ For Domain org ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0




2


100%|██████████| 200/200 [01:19<00:00,  2.53it/s]

++++++++++ For Domain iclr ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0
++++++++++ For Domain neur ++++++++++
Accuracy : 0.9930313588850174
Precision : 0.9927007299270073
Recall : 0.9927007299270073
F1-score : 0.9927007299270073
++++++++++ For Domain org ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0






In [119]:
soundness = soundness_models[1]
torch.save(soundness, f"{model_save_path}/soundness.pt")

### Combined:

In [12]:
def log_odds_aggregate(prob_list):
    logits = [np.log(p / (1 - p)) for p in prob_list]
    mean_logit = np.mean(logits)
    aggregated_prob = 1 / (1 + np.exp(-mean_logit))
    return aggregated_prob


In [13]:
vocab = json.load(open("vocabulary.json"))
len(vocab)

20983

In [14]:
ai_dict_combined = {}
human_dict_combined = {}

for word in vocab:
    v_probs = []
    for asp in list(ai_dict.keys()):
        if word in ai_dict[asp]:
            if ai_dict[asp][word]!=0:
                v_probs.append(ai_dict[asp][word])

    try:
        if len(v_probs):
            ai_dict_combined[word] = log_odds_aggregate(v_probs)
    except:
        print(word)

for word in vocab:
    v_probs = []
    for asp in list(human_dict.keys()):
        if word in human_dict[asp]:
            if human_dict[asp][word]!=0:
                v_probs.append(human_dict[asp][word])
    if len(v_probs):
        human_dict_combined[word] = log_odds_aggregate(v_probs)


the


In [15]:
ai_dict['combined'] = ai_dict_combined
human_dict['combined'] = human_dict_combined

In [ ]:
# json.dump(ai_dict_combined, open("ai_dict_combined.json","w"))
# json.dump(human_dict_combined, open("human_dict_combined.json","w"))

In [ ]:
best_params = {'num_hidden_layers': 2,
 'neurons_per_layer': 32,
 'epochs': 200,
 'learning_rate': 0.055825772063256424,
 'dropout_rate': 0.2,
 'batch_size': 16,
 'optimizer_name': 'RMSprop',
 'weight_decay': 0.0003307982403899191,
 "input_dim":2,
 "output_dim":1}

In [185]:
ai = {
    domain:[[probability_finder(revw, ai_dict_combined),probability_finder(revw, human_dict_combined)] for revw in list(data[domain]['ai'].values())] for domain in list(data.keys())
}

human = {
    domain:[[probability_finder(revw, ai_dict_combined),probability_finder(revw, human_dict_combined)] for revw in list(data[domain]['human'].values())] for domain in list(data.keys())
}

combined_models = []
for i in range(3):
    print(f"{i}")
    combined_models.append(model_trainer(ai, human, best_params))
    print("\n"*3)

0


100%|██████████| 200/200 [01:19<00:00,  2.53it/s]


++++++++++ For Domain iclr ++++++++++
Accuracy : 0.9966555183946488
Precision : 1.0
Recall : 0.9931506849315068
F1-score : 0.9965635738831615
++++++++++ For Domain neur ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0
++++++++++ For Domain org ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0




1


100%|██████████| 200/200 [01:23<00:00,  2.40it/s]


++++++++++ For Domain iclr ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0
++++++++++ For Domain neur ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0
++++++++++ For Domain org ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0




2


100%|██████████| 200/200 [01:18<00:00,  2.54it/s]

++++++++++ For Domain iclr ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0
++++++++++ For Domain neur ++++++++++
Accuracy : 0.9965156794425087
Precision : 1.0
Recall : 0.9935897435897436
F1-score : 0.9967845659163987
++++++++++ For Domain org ++++++++++
Accuracy : 0.975
Precision : 1.0
Recall : 0.9487179487179487
F1-score : 0.9736842105263158






In [ ]:
combined = combined_models[1]
# torch.save(combined, f"{model_save_path}/combined.pt")

In [16]:
# Combines the prob of all the 4 models and then predicts
models = {
    "soundness":torch.load(f"{model_save_path}/soundness.pt"),
    "originality":torch.load(f"{model_save_path}/originality.pt"),
    "motivation":torch.load(f"{model_save_path}/motivation.pt"),
    "clarity":torch.load(f"{model_save_path}/clarity.pt"),
    "combined":torch.load(f"{model_save_path}/combined.pt"),
}

In [192]:
models = {
    "soundness":soundness,
    "originality":originality,
    "motivation":motivation,
    "clarity":clarity,
    "combined":combined,
}

Combination can be done in two ways. 
- to add the aspects probability first and then train the model
- Or to take prob of each model, (just before the prediction) and then add to get a final prediction

## Training Models (Domain wise)

In [12]:
# The only difference of here from the before one will be how we are generating the data

In [13]:
folders = ["ai_iclr","ai_neur","ai_org","human_iclr","human_neur","human_org"]
path = "/Data/sandeep/Vardhan/Journal/Dataset/Aspects"
ai_iclr = process_aspects(get_data([folders[0]], path))

# def get_revw(revw_dict, asp):

In [14]:
def ultra_get_data_(path,domain):

    return {domain:ultra_get_data(path)[domain]}

In [15]:
aspects = {'Motivation/Impact':'motivation', 'Originality':'originality', 'Soundness/Correctness':'soundness', 'Clarity':'clarity'}
path = "/Data/sandeep/Vardhan/Journal/Dataset/Aspects"
model_save_path = "/Data/sandeep/Vardhan/Journal/models/domain"

# Normalize the data!!!

### Clarity

In [ ]:
# Just keep changing the domain in a loop, and everything will work finely

#### ICLR:

In [16]:
data = ultra_get_data_(path,'iclr')

In [17]:
ASP = list(aspects.values())[3]
ai = {
    domain:[[probability_finder(revw, ai_dict[ASP]),probability_finder(revw, human_dict[ASP])] for revw in list(data[domain]['ai'].values())] for domain in list(data.keys())
}

human = {
    domain:[[probability_finder(revw, ai_dict[ASP]),probability_finder(revw, human_dict[ASP])] for revw in list(data[domain]['human'].values())] for domain in list(data.keys())
}

In [30]:
mean = yup.mean(dim=1, keepdim=True)  # shape: (num_samples, 1)
std = yup.std(dim=1, unbiased=False, keepdim=True)  # shape: (num_samples, 1)

# Standardize per sample
standardized_yup = (yup - mean) / std

In [28]:
yup.mean(dim=1).shape, yup.shape

(torch.Size([760]), torch.Size([760, 2]))

In [33]:
standardized_yup.tolist()

[[1.0000003576278687, -0.9999996423721313],
 [1.0, -1.0],
 [0.9999995231628418, -1.0000004768371582],
 [1.000001072883606, -0.999998927116394],
 [1.0, -1.0],
 [1.0000004768371582, -0.9999995827674866],
 [1.0, -1.0],
 [1.0, -1.0],
 [1.0, -1.0],
 [1.0, -1.0],
 [0.9999992251396179, -1.0000008344650269],
 [1.0000022649765015, -0.9999977946281433],
 [1.0, -1.0],
 [0.999999463558197, -1.0000004768371582],
 [1.0, -1.0],
 [1.0, -1.0],
 [1.0, -1.0],
 [1.0000004768371582, -0.9999995231628418],
 [1.0, -1.0],
 [1.0, -1.0],
 [1.0, -1.0],
 [1.000000238418579, -0.9999997615814209],
 [-1.0, 1.0],
 [1.0, -1.0],
 [0.9999997019767761, -1.0000003576278687],
 [1.0, -1.0],
 [1.0, -1.0],
 [1.0, -1.0],
 [1.0, -1.0],
 [1.0, -1.0],
 [1.0, -1.0],
 [0.9999972581863403, -1.0000027418136597],
 [1.0, -1.0],
 [1.0, -1.0],
 [-0.9999902844429016, 1.0000096559524536],
 [0.9999987483024597, -1.0000011920928955],
 [1.0, -1.0],
 [0.9999995231628418, -1.0000004768371582],
 [1.0000005960464478, -0.9999994039535522],
 [1.0, -

In [46]:
clarity_models = []
for i in range(5):
    print(f"{i}")
    clarity_models.append(model_trainer(ai, human, best_params))
    print("\n"*3)

0


  0%|          | 0/150 [00:00<?, ?it/s]

100%|██████████| 150/150 [00:26<00:00,  5.62it/s]


++++++++++ For Domain iclr ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0




1


100%|██████████| 150/150 [00:26<00:00,  5.64it/s]


...

++++++++++ For Domain iclr ++++++++++
Accuracy : 0.9966555183946488
Precision : 1.0
Recall : 0.9933774834437086
F1-score : 0.9966777408637874




3


100%|██████████| 150/150 [00:26<00:00,  5.68it/s]


++++++++++ For Domain iclr ++++++++++
Accuracy : 0.8294314381270903
Precision : 1.0
Recall : 0.6688311688311688
F1-score : 0.8015564202334631




4


100%|██████████| 150/150 [00:26<00:00,  5.71it/s]

++++++++++ For Domain iclr ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0






In [ ]:
torch.save(clarity_models[0], f"{model_save_path}/clarity.pt")

In [ ]:
clarity = clarity_models[0]

In [ ]:
yup = torch.load(f"{model_save_path}/clarity.pt")

### Motivation

In [ ]:
best_params = {'num_hidden_layers': 3,
 'neurons_per_layer': 32,
 'epochs': 150,
 'learning_rate': 0.055825772063256424,
 'dropout_rate': 0.2,
 'batch_size': 16,
 'optimizer_name': 'RMSprop',
 'weight_decay': 0.0003307982403899191,
 "input_dim":2,
 "output_dim":1}


In [ ]:
ASP = list(aspects.values())[0]
data = ultra_get_data(path,list(aspects.keys())[0])
ai = {
    domain:[[probability_finder(revw, ai_dict[ASP]),probability_finder(revw, human_dict[ASP])] for revw in list(data[domain]['ai'].values())] for domain in list(data.keys())
}

human = {
    domain:[[probability_finder(revw, ai_dict[ASP]),probability_finder(revw, human_dict[ASP])] for revw in list(data[domain]['human'].values())] for domain in list(data.keys())
}

motivation_models = []
for i in range(5):
    print(f"{i}")
    motivation_models.append(model_trainer(ai, human, best_params))
    print("\n"*3)
# torch.save(model, f"{model_save_path}/motivation.pt")

0


100%|██████████| 150/150 [01:15<00:00,  1.99it/s]


++++++++++ For Domain iclr ++++++++++
Accuracy : 0.9966555183946488
Precision : 1.0
Recall : 0.9932432432432432
F1-score : 0.9966101694915255
++++++++++ For Domain neur ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0
++++++++++ For Domain org ++++++++++
Accuracy : 0.9625
Precision : 1.0
Recall : 0.9387755102040817
F1-score : 0.968421052631579




1


100%|██████████| 150/150 [01:15<00:00,  1.99it/s]


++++++++++ For Domain iclr ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0
++++++++++ For Domain neur ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0
++++++++++ For Domain org ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0




2


...

++++++++++ For Domain iclr ++++++++++
Accuracy : 0.9966555183946488
Precision : 1.0
Recall : 0.9933333333333333
F1-score : 0.9966555183946488
++++++++++ For Domain neur ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0
++++++++++ For Domain org ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0




3


100%|██████████| 150/150 [01:15<00:00,  1.99it/s]


++++++++++ For Domain iclr ++++++++++
Accuracy : 0.9899665551839465
Precision : 0.9870967741935484
Recall : 0.9935064935064936
F1-score : 0.9902912621359223
++++++++++ For Domain neur ++++++++++
Accuracy : 0.9825783972125436
Precision : 0.9695121951219512
Recall : 1.0
F1-score : 0.9845201238390093
++++++++++ For Domain org ++++++++++
Accuracy : 0.9875
Precision : 0.972972972972973
Recall : 1.0
F1-score : 0.9863013698630136




4


100%|██████████| 150/150 [01:15<00:00,  1.99it/s]

++++++++++ For Domain iclr ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0
++++++++++ For Domain neur ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0
++++++++++ For Domain org ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0






In [ ]:
motivation = motivation_models[-1]
torch.save(motivation, f"{model_save_path}/motivation.pt")

In [ ]:
# ASP = aspects[0]
# ai = {
#     domain:[[probability_finder(revw, ai_dict[ASP]),probability_finder(revw, human_dict[ASP])] for revw in list(data[domain]['ai'].values())] for domain in list(data.keys())
# }

# human = {
#     domain:[[probability_finder(revw, ai_dict[ASP]),probability_finder(revw, human_dict[ASP])] for revw in list(data[domain]['human'].values())] for domain in list(data.keys())
# }

# motivation = model_trainer(ai, human, best_params)
# torch.save(model, f"{model_save_path}/motivation.pt")

### Originality

In [ ]:
best_params = {'num_hidden_layers': 3,
 'neurons_per_layer': 32,
 'epochs': 300,
 'learning_rate': 0.055825772063256424,
 'dropout_rate': 0.2,
 'batch_size': 16,
 'optimizer_name': 'RMSprop',
 'weight_decay': 0.0003307982403899191,
 "input_dim":2,
 "output_dim":1}

In [ ]:
ASP = list(aspects.values())[1]
data = ultra_get_data(path,list(aspects.keys())[1])
ai = {
    domain:[[probability_finder(revw, ai_dict[ASP]),probability_finder(revw, human_dict[ASP])] for revw in list(data[domain]['ai'].values())] for domain in list(data.keys())
}

human = {
    domain:[[probability_finder(revw, ai_dict[ASP]),probability_finder(revw, human_dict[ASP])] for revw in list(data[domain]['human'].values())] for domain in list(data.keys())
}

originality_models = []

for i in range(5):
    print(f"{i}")
    originality_models.append(model_trainer(ai, human, best_params))
    print("\n"*3)
# torch.save(model, f"{model_save_path}/originality.pt")

0


100%|██████████| 300/300 [02:30<00:00,  1.99it/s]


++++++++++ For Domain iclr ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0
++++++++++ For Domain neur ++++++++++
Accuracy : 0.9651567944250871
Precision : 1.0
Recall : 0.9295774647887324
F1-score : 0.9635036496350365
++++++++++ For Domain org ++++++++++
Accuracy : 0.9625
Precision : 1.0
Recall : 0.9302325581395349
F1-score : 0.963855421686747




1


100%|██████████| 300/300 [02:31<00:00,  1.99it/s]


++++++++++ For Domain iclr ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0
++++++++++ For Domain neur ++++++++++
Accuracy : 0.9930313588850174
Precision : 1.0
Recall : 0.9862068965517241
F1-score : 0.9930555555555556
++++++++++ For Domain org ++++++++++
Accuracy : 0.95
Precision : 1.0
Recall : 0.9090909090909091
F1-score : 0.9523809523809523




2


...

++++++++++ For Domain iclr ++++++++++
Accuracy : 0.9966555183946488
Precision : 1.0
Recall : 0.9937106918238994
F1-score : 0.9968454258675079
++++++++++ For Domain neur ++++++++++
Accuracy : 0.975609756097561
Precision : 1.0
Recall : 0.951048951048951
F1-score : 0.974910394265233
++++++++++ For Domain org ++++++++++
Accuracy : 0.95
Precision : 1.0
Recall : 0.9
F1-score : 0.9473684210526315




3


100%|██████████| 300/300 [02:29<00:00,  2.01it/s]


++++++++++ For Domain iclr ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0
++++++++++ For Domain neur ++++++++++
Accuracy : 0.9895470383275261
Precision : 1.0
Recall : 0.9765625
F1-score : 0.9881422924901185
++++++++++ For Domain org ++++++++++
Accuracy : 0.875
Precision : 1.0
Recall : 0.7560975609756098
F1-score : 0.8611111111111112




4


100%|██████████| 300/300 [02:30<00:00,  1.99it/s]

++++++++++ For Domain iclr ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0
++++++++++ For Domain neur ++++++++++
Accuracy : 0.9721254355400697
Precision : 1.0
Recall : 0.9436619718309859
F1-score : 0.9710144927536232
++++++++++ For Domain org ++++++++++
Accuracy : 0.95
Precision : 1.0
Recall : 0.9024390243902439
F1-score : 0.9487179487179487






In [ ]:
# originality = originality_models[1]
# torch.save(originality, f"{model_save_path}/originality.pt")

### Soundness:

In [ ]:
best_params = {'num_hidden_layers': 2,
 'neurons_per_layer': 32,
 'epochs': 200,
 'learning_rate': 0.055825772063256424,
 'dropout_rate': 0.2,
 'batch_size': 16,
 'optimizer_name': 'RMSprop',
 'weight_decay': 0.0003307982403899191,
 "input_dim":2,
 "output_dim":1}


In [ ]:
ASP = list(aspects.values())[2]
data = ultra_get_data(path,list(aspects.keys())[2])
ai = {
    domain:[[probability_finder(revw, ai_dict[ASP]),probability_finder(revw, human_dict[ASP])] for revw in list(data[domain]['ai'].values())] for domain in list(data.keys())
}

human = {
    domain:[[probability_finder(revw, ai_dict[ASP]),probability_finder(revw, human_dict[ASP])] for revw in list(data[domain]['human'].values())] for domain in list(data.keys())
}

soundness_models = []
for i in range(3):
    print(f"{i}")
    soundness_models.append(model_trainer(ai, human, best_params))
    print("\n"*3)
# torch.save(model, f"{model_save_path}/soundness.pt")

0


100%|██████████| 200/200 [01:18<00:00,  2.56it/s]


++++++++++ For Domain iclr ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0
++++++++++ For Domain neur ++++++++++
Accuracy : 0.9965156794425087
Precision : 1.0
Recall : 0.993421052631579
F1-score : 0.9966996699669967
++++++++++ For Domain org ++++++++++
Accuracy : 0.975
Precision : 1.0
Recall : 0.9444444444444444
F1-score : 0.9714285714285714




1


100%|██████████| 200/200 [01:19<00:00,  2.50it/s]


++++++++++ For Domain iclr ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0
++++++++++ For Domain neur ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0
++++++++++ For Domain org ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0




2


100%|██████████| 200/200 [01:19<00:00,  2.53it/s]

++++++++++ For Domain iclr ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0
++++++++++ For Domain neur ++++++++++
Accuracy : 0.9930313588850174
Precision : 0.9927007299270073
Recall : 0.9927007299270073
F1-score : 0.9927007299270073
++++++++++ For Domain org ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0






In [ ]:
soundness = soundness_models[1]
torch.save(soundness, f"{model_save_path}/soundness.pt")

### Combined:

In [ ]:
def log_odds_aggregate(prob_list):
    logits = [np.log(p / (1 - p)) for p in prob_list]
    mean_logit = np.mean(logits)
    aggregated_prob = 1 / (1 + np.exp(-mean_logit))
    return aggregated_prob


In [ ]:
vocab = json.load(open("vocabulary.json"))
len(vocab)

20983

In [ ]:
ai_dict_combined = {}
human_dict_combined = {}

for word in vocab:
    v_probs = []
    for asp in list(ai_dict.keys()):
        if word in ai_dict[asp]:
            if ai_dict[asp][word]!=0:
                v_probs.append(ai_dict[asp][word])

    try:
        if len(v_probs):
            ai_dict_combined[word] = log_odds_aggregate(v_probs)
    except:
        print(word)

for word in vocab:
    v_probs = []
    for asp in list(human_dict.keys()):
        if word in human_dict[asp]:
            if human_dict[asp][word]!=0:
                v_probs.append(human_dict[asp][word])
    if len(v_probs):
        human_dict_combined[word] = log_odds_aggregate(v_probs)


the


In [ ]:
ai_dict['combined'] = ai_dict_combined
human_dict['combined'] = human_dict_combined

In [ ]:
# json.dump(ai_dict_combined, open("ai_dict_combined.json","w"))
# json.dump(human_dict_combined, open("human_dict_combined.json","w"))

In [ ]:
best_params = {'num_hidden_layers': 2,
 'neurons_per_layer': 32,
 'epochs': 200,
 'learning_rate': 0.055825772063256424,
 'dropout_rate': 0.2,
 'batch_size': 16,
 'optimizer_name': 'RMSprop',
 'weight_decay': 0.0003307982403899191,
 "input_dim":2,
 "output_dim":1}

In [ ]:
ai = {
    domain:[[probability_finder(revw, ai_dict_combined),probability_finder(revw, human_dict_combined)] for revw in list(data[domain]['ai'].values())] for domain in list(data.keys())
}

human = {
    domain:[[probability_finder(revw, ai_dict_combined),probability_finder(revw, human_dict_combined)] for revw in list(data[domain]['human'].values())] for domain in list(data.keys())
}

combined_models = []
for i in range(3):
    print(f"{i}")
    combined_models.append(model_trainer(ai, human, best_params))
    print("\n"*3)

0


100%|██████████| 200/200 [01:19<00:00,  2.53it/s]


++++++++++ For Domain iclr ++++++++++
Accuracy : 0.9966555183946488
Precision : 1.0
Recall : 0.9931506849315068
F1-score : 0.9965635738831615
++++++++++ For Domain neur ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0
++++++++++ For Domain org ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0




1


100%|██████████| 200/200 [01:23<00:00,  2.40it/s]


++++++++++ For Domain iclr ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0
++++++++++ For Domain neur ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0
++++++++++ For Domain org ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0




2


100%|██████████| 200/200 [01:18<00:00,  2.54it/s]

++++++++++ For Domain iclr ++++++++++
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
F1-score : 1.0
++++++++++ For Domain neur ++++++++++
Accuracy : 0.9965156794425087
Precision : 1.0
Recall : 0.9935897435897436
F1-score : 0.9967845659163987
++++++++++ For Domain org ++++++++++
Accuracy : 0.975
Precision : 1.0
Recall : 0.9487179487179487
F1-score : 0.9736842105263158






In [ ]:
combined = combined_models[1]
# torch.save(combined, f"{model_save_path}/combined.pt")

In [ ]:
# Combines the prob of all the 4 models and then predicts
models = {
    "soundness":torch.load(f"{model_save_path}/soundness.pt"),
    "originality":torch.load(f"{model_save_path}/originality.pt"),
    "motivation":torch.load(f"{model_save_path}/motivation.pt"),
    "clarity":torch.load(f"{model_save_path}/clarity.pt"),
    "combined":torch.load(f"{model_save_path}/combined.pt"),
}

In [ ]:
models = {
    "soundness":soundness,
    "originality":originality,
    "motivation":motivation,
    "clarity":clarity,
    "combined":combined,
}

Combination can be done in two ways. 
- to add the aspects probability first and then train the model
- Or to take prob of each model, (just before the prediction) and then add to get a final prediction

In [19]:
model_save_path = "/Data/sandeep/Vardhan/Journal/models/domain"

## Testing

### Checker:

To run all the models on some dataset

In [9]:
models = {
    "soundness":torch.load(f"{model_save_path}/soundness.pt"),
    "originality":torch.load(f"{model_save_path}/originality.pt"),
    "motivation":torch.load(f"{model_save_path}/motivation.pt"),
    "clarity":torch.load(f"{model_save_path}/clarity.pt"),
}

In [121]:
def log_odds_aggregate(prob_list):
    logits = [np.log(p / (1 - p)) for p in prob_list]
    mean_logit = np.mean(logits)
    aggregated_prob = 1 / (1 + np.exp(-mean_logit))
    return aggregated_prob

probs = [0.8, 0.6, 0.9]
print(log_odds_aggregate(probs))

0.7907846123994696


In [122]:
def log_odds_aggregate_torch(p1, p2,p3,p4):
    # Convert to log-odds
    logits1 = torch.log(p1 / (1 - p1))
    logits2 = torch.log(p2 / (1 - p2))
    logits3 = torch.log(p3 / (1 - p3))
    logits4 = torch.log(p4 / (1 - p4))
    
    # Average logits
    mean_logits = torch.mean(logits1 + logits2+logits3 + logits4, dim=1)
    
    # Convert back to probability
    aggregated_probs = torch.sigmoid(mean_logits)
    return aggregated_probs

In [17]:
def score_finder(true_vals, pred_vals):
    accuracy = accuracy_score(true_vals, pred_vals)
    precision = precision_score(true_vals, pred_vals, average='binary')
    recall = recall_score(true_vals, pred_vals, average='binary')
    f1 = f1_score(true_vals, pred_vals, average='binary')

    print(f"Accuracy : {accuracy}")
    print(f"Precision : {precision}")
    print(f"Recall : {recall}")
    print(f"F1-score : {f1}")

In [18]:
def tester_all_models(data,models):
    # model_logits = {}
    for model_id, model in models.items():
        # model_logits[model_id] = {}
        print(f"\n\n {'+'*10} {model_id} {'+'*10}")

        ASP = model_id

        ai = {
            domain:[[probability_finder(revw, ai_dict[ASP]),probability_finder(revw, human_dict[ASP])] for revw in list(data[domain]['ai'].values())] for domain in list(data.keys())
        }

        human = {
            domain:[[probability_finder(revw, ai_dict[ASP]),probability_finder(revw, human_dict[ASP])] for revw in list(data[domain]['human'].values())] for domain in list(data.keys())
        }
        
        for domain_id in list(ai.keys()):
            print(f"\n {'+'*10} {domain_id} {'+'*10}")
            X_test = torch.tensor(ai[domain_id] + human[domain_id],dtype=torch.float32).to(device)
            y_test = torch.tensor([[1] for _ in range(len(ai[domain_id]))] + [[0] for _ in range(len(human[domain_id]))], dtype=torch.float32)

            # model_logits[model_id][domain_id] = torch.sigmoid(model(X_test.to(device)))

            test_dataset = CustomDataset(X_test, y_test)

            # Start the testing
            tester(model,test_dataset, device=device)


    # Make the combination model logic later on
    # Combined model have to be done at the last
    # for domain_id in list(model_logits['clarity'].keys()):
        # logits = log_odds_aggregate_torch(models['clarity'][domain_id])

We don't have to paraphrase the whole dataset, but just those reviews which we are going to test

In [5]:
print("e")

e


In [27]:
aspects

['motivation', 'originality', 'soundness', 'clarity']

In [28]:
ASP = aspects[3]
ai = {
    domain:[[probability_finder(revw, ai_dict[ASP]),probability_finder(revw, human_dict[ASP])] for revw in list(data[domain]['ai'].values())] for domain in list(data.keys())
}

human = {
    domain:[[probability_finder(revw, ai_dict[ASP]),probability_finder(revw, human_dict[ASP])] for revw in list(data[domain]['human'].values())] for domain in list(data.keys())
}

In [31]:
domain_id = 'iclr'
X_test = torch.tensor(ai[domain_id] + human[domain_id],dtype=torch.float32).to(device)
y_test = torch.tensor([[1] for _ in range(len(ai[domain_id]))] + [[0] for _ in range(len(human[domain_id]))], dtype=torch.float32)

yup1 = torch.sigmoid(models['clarity'](X_test.to(device)))
yup2 = torch.sigmoid(models['clarity'](X_test.to(device)))

#### Gemini Paraphrase:

In [37]:
test_keys = json.load(open("/Data/sandeep/Vardhan/Journal/Dataset/test_keys.json"))
def get_data(path, level,domain):
    data={}
    for f in test_keys[domain]:
        try:
            revw = json.load(open(os.path.join(path,f"{f}.json")))
            data[f] = revw[level]
        except:
            pass
            # print(f"Error at {file} in folder {f}")

    return data

In [38]:
folders = ["ai_iclr","ai_neur","ai_org","human_iclr","human_neur","human_org"]
path = "/Data/sandeep/Vardhan/Journal/Dataset/Aspects/gemini_paraphrase"

In [39]:
def para_tester(path):
    levels = ['low','med','high']

    for level in levels:
        ai_iclr = get_data(f"{path}/{folders[0]}", level,domain='iclr')
        ai_neur = get_data(f"{path}/{folders[1]}", level,domain='neur')
        ai_org = get_data(f"{path}/{folders[2]}", level,domain='org')
        human_iclr = get_data(f"{path}/{folders[3]}", level,domain='iclr')
        human_neur = get_data(f"{path}/{folders[4]}", level,domain='neur')
        human_org = get_data(f"{path}/{folders[5]}", level,domain='org')

        data = {
            "iclr":{"ai":ai_iclr ,
            "human":human_iclr} ,
            "neur":{"ai":ai_neur ,
            "human":human_neur },
            "org":{"ai":ai_org ,
            "human":human_org },
        }

        print("\n"*5,f"{'+'*10} For level {level} {'+'*10}")
        tester_all_models(data, models)

In [40]:
para_tester(path)






 ++++++++++ For level low ++++++++++


 ++++++++++ soundness ++++++++++

 ++++++++++ iclr ++++++++++
Accuracy : 0.9572368421052632
Precision : 0.9929078014184397
Recall : 0.9210526315789473
F1-score : 0.9556313993174061

 ++++++++++ neur ++++++++++
Accuracy : 0.90625
Precision : 0.9834710743801653
Recall : 0.8263888888888888
F1-score : 0.8981132075471698

 ++++++++++ org ++++++++++
Accuracy : 0.8375
Precision : 1.0
Recall : 0.675
F1-score : 0.8059701492537313


 ++++++++++ originality ++++++++++

 ++++++++++ iclr ++++++++++
Accuracy : 0.9506578947368421
Precision : 1.0
Recall : 0.9013157894736842
F1-score : 0.9480968858131488

 ++++++++++ neur ++++++++++
Accuracy : 0.8055555555555556
Precision : 0.9782608695652174
Recall : 0.625
F1-score : 0.7627118644067796

 ++++++++++ org ++++++++++
Accuracy : 0.6625
Precision : 1.0
Recall : 0.325
F1-score : 0.49056603773584906


 ++++++++++ motivation ++++++++++

 ++++++++++ iclr ++++++++++
Accuracy : 0.9473684210526315
Precision : 1.0
Recall 

In [32]:
tester_all_models( data,models)



 ++++++++++ soundness ++++++++++

 ++++++++++ iclr ++++++++++
Accuracy : 0.7368421052631579
Precision : 0.6551724137931034
Recall : 1.0
F1-score : 0.7916666666666666

 ++++++++++ neur ++++++++++
Accuracy : 0.7291666666666666
Precision : 0.6486486486486487
Recall : 1.0
F1-score : 0.7868852459016393

 ++++++++++ org ++++++++++
Accuracy : 0.975
Precision : 0.9523809523809523
Recall : 1.0
F1-score : 0.975609756097561


 ++++++++++ originality ++++++++++

 ++++++++++ iclr ++++++++++
Accuracy : 0.7861842105263158
Precision : 0.7004608294930875
Recall : 1.0
F1-score : 0.8238482384823849

 ++++++++++ neur ++++++++++
Accuracy : 0.7638888888888888
Precision : 0.6792452830188679
Recall : 1.0
F1-score : 0.8089887640449438

 ++++++++++ org ++++++++++
Accuracy : 0.95
Precision : 1.0
Recall : 0.9
F1-score : 0.9473684210526315


 ++++++++++ motivation ++++++++++

 ++++++++++ iclr ++++++++++
Accuracy : 0.8256578947368421
Precision : 0.7414634146341463
Recall : 1.0
F1-score : 0.8515406162464986

 ++++

In [132]:
tester_all_models( models)



 ++++++++++ soundness ++++++++++

 ++++++++++ iclr ++++++++++
Accuracy : 0.7401315789473685
Precision : 0.6585577758470895
Recall : 0.9973684210526316
F1-score : 0.7933019361590791

 ++++++++++ neur ++++++++++
Accuracy : 0.7229166666666667
Precision : 0.6439461883408072
Recall : 0.9972222222222222
F1-score : 0.7825613079019074

 ++++++++++ org ++++++++++
Accuracy : 0.9475
Precision : 0.949748743718593
Recall : 0.945
F1-score : 0.9473684210526315


 ++++++++++ originality ++++++++++

 ++++++++++ iclr ++++++++++
Accuracy : 0.7894736842105263
Precision : 0.7071563088512242
Recall : 0.9881578947368421
F1-score : 0.8243688254665203

 ++++++++++ neur ++++++++++
Accuracy : 0.7805555555555556
Precision : 0.7044534412955465
Recall : 0.9666666666666667
F1-score : 0.8149882903981265

 ++++++++++ org ++++++++++
Accuracy : 0.73
Precision : 0.9693877551020408
Recall : 0.475
F1-score : 0.6375838926174496


 ++++++++++ motivation ++++++++++

 ++++++++++ iclr ++++++++++
Accuracy : 0.8256578947368421


In [34]:
X = {id:ai[id]+human[id] for id in list(ai.keys())}
y = {id:[[1] for _ in range(len(ai[id]))] + [[0] for _ in range(len(human[id]))] for id in list(ai.keys())}

data = {id:train_test_split(X[id], y[id], test_size=0.2) for id in list(X.keys())}
test_data = {id:{'X':data[id][1], 'y':data[id][3]} for id in list(data.keys())}
id = 'iclr'
X_test = torch.tensor(test_data[id]['X'], dtype=torch.float32)
y_test = torch.tensor(test_data[id]['y'], dtype=torch.float32)

In [87]:
test_keys = json.load(open("Dataset/test_keys.json"))
len(test_keys['iclr']),len(test_keys['neur']),len(test_keys['org'])

(152, 144, 40)

In [88]:
test_keys.keys()

dict_keys(['iclr', 'neur', 'org'])

#### GPT Paraphraser:

In [33]:
def get_data(path, level):
    data={}
    for f in os.listdir(path):
        try:
            revw = json.load(open(os.path.join(path,f)))
            # data[f[:-5]] = do_eval(revw[level])
            revw = do_eval(revw[level])

            processed_revw = ""
            for asp in list(revw.keys()):
                processed_revw += " ".join(revw[asp])
            data[f[:-5]] = processed_revw
        except:
            revw = json.load(open(os.path.join(path,f)))
            revw = do_eval(revw[level])
            if type(revw)==str:
                data[f[:-5]] = revw
        #     pass
            else:
                print(f"Error at {f} in folder {f}")

    return data

In [34]:
folders = ["ai_iclr","ai_neur","ai_org","human_iclr","human_neur","human_org"]
path = "/Data/sandeep/Vardhan/Journal/Dataset/Aspects/paraphrase"

In [35]:
def gpt_para_tester(path):
    levels = ['low','med','high']

    for level in levels:
        ai_iclr = get_data(f"{path}/{folders[0]}", level)
        ai_neur = get_data(f"{path}/{folders[1]}", level)
        ai_org = get_data(f"{path}/{folders[2]}", level)
        human_iclr = get_data(f"{path}/{folders[3]}", level)
        human_neur = get_data(f"{path}/{folders[4]}", level)
        human_org = get_data(f"{path}/{folders[5]}", level)

        data = {
            "iclr":{"ai":ai_iclr ,
            "human":human_iclr} ,
            "neur":{"ai":ai_neur ,
            "human":human_neur },
            "org":{"ai":ai_org ,
            "human":human_org },
        }

        print("\n"*5,f"{'+'*10} For level {level} {'+'*10}")
        tester_all_models(data, models)

In [36]:
gpt_para_tester(path)






 ++++++++++ For level low ++++++++++


 ++++++++++ soundness ++++++++++

 ++++++++++ iclr ++++++++++
Accuracy : 0.9879437374413932
Precision : 0.9830729166666666
Recall : 0.993421052631579
F1-score : 0.9882198952879581

 ++++++++++ neur ++++++++++
Accuracy : 0.9797768479776848
Precision : 0.976551724137931
Recall : 0.9833333333333333
F1-score : 0.9799307958477509

 ++++++++++ org ++++++++++
Accuracy : 0.98
Precision : 1.0
Recall : 0.96
F1-score : 0.9795918367346939


 ++++++++++ originality ++++++++++

 ++++++++++ iclr ++++++++++
Accuracy : 0.9879437374413932
Precision : 0.9933510638297872
Recall : 0.9828947368421053
F1-score : 0.9880952380952381

 ++++++++++ neur ++++++++++
Accuracy : 0.9449093444909344
Precision : 0.9938366718027735
Recall : 0.8958333333333334
F1-score : 0.9422936449963477

 ++++++++++ org ++++++++++
Accuracy : 0.8225
Precision : 1.0
Recall : 0.645
F1-score : 0.78419452887538


 ++++++++++ motivation ++++++++++

 ++++++++++ iclr ++++++++++
Accuracy : 0.990622906

In [15]:
# import json
xmx = json.load(open("/Data/sandeep/Vardhan/Journal/Dataset/Aspects/multiple_paraphrase/ai_iclr/1/famc03Gg231.json"))
len(xmx)

2242

In [203]:
gpt_para_tester(path)






 ++++++++++ For level low ++++++++++


 ++++++++++ soundness ++++++++++

 ++++++++++ iclr ++++++++++
Accuracy : 0.9879437374413932
Precision : 0.9830729166666666
Recall : 0.993421052631579
F1-score : 0.9882198952879581

 ++++++++++ neur ++++++++++
Accuracy : 0.9797768479776848
Precision : 0.976551724137931
Recall : 0.9833333333333333
F1-score : 0.9799307958477509

 ++++++++++ org ++++++++++
Accuracy : 0.98
Precision : 1.0
Recall : 0.96
F1-score : 0.9795918367346939


 ++++++++++ originality ++++++++++

 ++++++++++ iclr ++++++++++
Accuracy : 0.9879437374413932
Precision : 0.9933510638297872
Recall : 0.9828947368421053
F1-score : 0.9880952380952381

 ++++++++++ neur ++++++++++
Accuracy : 0.9449093444909344
Precision : 0.9938366718027735
Recall : 0.8958333333333334
F1-score : 0.9422936449963477

 ++++++++++ org ++++++++++
Accuracy : 0.8225
Precision : 1.0
Recall : 0.645
F1-score : 0.78419452887538


 ++++++++++ motivation ++++++++++

 ++++++++++ iclr ++++++++++
Accuracy : 0.990622906

#### Sentence Position Change:

In [210]:
path = "/Data/sandeep/Vardhan/Journal/Dataset/Aspects/sentence_pos"

def get_simple_data(path):
    ai_iclr = json.load(open(f"{path}/ai_iclr.json"))
    ai_neur = json.load(open(f"{path}/ai_neur.json"))
    ai_org = json.load(open(f"{path}/ai_org.json"))
    human_iclr = json.load(open(f"{path}/human_iclr.json"))
    human_neur = json.load(open(f"{path}/human_neur.json"))
    human_org = json.load(open(f"{path}/human_org.json"))
    data = {
                "iclr":{"ai":ai_iclr ,
                "human":human_iclr} ,
                "neur":{"ai":ai_neur ,
                "human":human_neur },
                "org":{"ai":ai_org ,
                "human":human_org },
            }
    
    return data

data = get_simple_data(path)

In [211]:
tester_all_models(data,models)



 ++++++++++ soundness ++++++++++

 ++++++++++ iclr ++++++++++
Accuracy : 0.9979906229068989
Precision : 0.9973718791064389
Recall : 0.9986842105263158
F1-score : 0.9980276134122288

 ++++++++++ neur ++++++++++
Accuracy : 0.9944211994421199
Precision : 0.989010989010989
Recall : 1.0
F1-score : 0.994475138121547

 ++++++++++ org ++++++++++
Accuracy : 0.9975
Precision : 1.0
Recall : 0.995
F1-score : 0.9974937343358395


 ++++++++++ originality ++++++++++

 ++++++++++ iclr ++++++++++
Accuracy : 0.999330207635633
Precision : 1.0
Recall : 0.9986842105263158
F1-score : 0.9993416721527321

 ++++++++++ neur ++++++++++
Accuracy : 0.9944211994421199
Precision : 0.9972067039106145
Recall : 0.9916666666666667
F1-score : 0.9944289693593314

 ++++++++++ org ++++++++++
Accuracy : 0.9475
Precision : 1.0
Recall : 0.895
F1-score : 0.9445910290237467


 ++++++++++ motivation ++++++++++

 ++++++++++ iclr ++++++++++
Accuracy : 0.999330207635633
Precision : 1.0
Recall : 0.9986842105263158
F1-score : 0.9993

#### Synonym Change attack

In [212]:
path = "/Data/sandeep/Vardhan/Journal/Dataset/Aspects/synonym_attack"
data = get_simple_data(path)

In [213]:
tester_all_models(data,models)



 ++++++++++ soundness ++++++++++

 ++++++++++ iclr ++++++++++
Accuracy : 0.9979906229068989
Precision : 0.9986824769433466
Recall : 0.9973684210526316
F1-score : 0.9980250164581962

 ++++++++++ neur ++++++++++
Accuracy : 0.997907949790795
Precision : 0.9972260748959778
Recall : 0.9986111111111111
F1-score : 0.9979181124219292

 ++++++++++ org ++++++++++
Accuracy : 0.995
Precision : 1.0
Recall : 0.99
F1-score : 0.9949748743718593


 ++++++++++ originality ++++++++++

 ++++++++++ iclr ++++++++++
Accuracy : 0.999330207635633
Precision : 1.0
Recall : 0.9986842105263158
F1-score : 0.9993416721527321

 ++++++++++ neur ++++++++++
Accuracy : 0.9846582984658299
Precision : 1.0
Recall : 0.9694444444444444
F1-score : 0.9844851904090268

 ++++++++++ org ++++++++++
Accuracy : 0.9775
Precision : 1.0
Recall : 0.955
F1-score : 0.9769820971867008


 ++++++++++ motivation ++++++++++

 ++++++++++ iclr ++++++++++
Accuracy : 0.999330207635633
Precision : 1.0
Recall : 0.9986842105263158
F1-score : 0.99934

In [70]:
yup = json.load(open("/Data/sandeep/Vardhan/Journal/Dataset/Aspects/paraphrase/ai_iclr/hC474P6AqN-.json"))
yup['high']

'{\n  \'Motivation/Impact\': \'```json\\n[\\n  "This study introduces a multitype continuous disentanglement variational autoencoder designed to solve the problem of mismatched labeling systems in machine learning datasets when integrating various datasets that have been annotated with different categorical frameworks.",\\n  "Tests performed on synthetic datasets reveal a significant relationship between the disentangled latent values and the actual generative factors, indicating promising potential for enhancing explainability within neural network architectures.",\\n  "The innovative multitype continuous disentanglement variational autoencoder proposed here provides valuable solutions for merging datasets that utilize differing categorical models by recognizing and separating the foundational generative factors.",\\n  "The paper clearly articulates the shortcomings of current methods and substantiates the necessity for a fresh approach informed by the model put forward.",\\n  "This t

In [ ]:
path = "/Data/sandeep/Vardhan/Journal/Dataset/Aspects/paraphrase"
data = ultra_get_data(path)

KeyError: 0

In [90]:
len(data['iclr'][2])

1194

In [84]:
len(X_test),len(y_test)

(299, 1194)

In [47]:
probability_finder(ai_iclr['fM8VzFD_2-']['review'], ai_dict['clarity'])

13.939189189189197

In [48]:
ai_dict.keys()

dict_keys(['motivation', 'originality', 'soundness', 'clarity'])

#### Gemini Multiple Paraphrase:

##### Paraphrase 1:

In [20]:
folders = ["ai_iclr","ai_neur","ai_org","human_iclr","human_neur","human_org"]

In [25]:
folder_path = "/Data/sandeep/Vardhan/Journal/Dataset/Aspects/multiple_paraphrase"

def get_data(path,level):
    data_ = {}

    for f in folders:
        data_[f] = {}
        p =f"{folder_path}/{f}/{level}/"
        for id in os.listdir(p):
            data_[f][id[:-5]] = json.load(open(f"{p}{id}"))

    data = {
            "iclr":{"ai":data_["ai_iclr"] ,
            "human":data_["human_iclr"]} ,
            "neur":{"ai":data_["ai_neur"] ,
            "human":data_["human_neur"] },
            "org":{"ai":data_["ai_org"] ,
            "human":data_["human_org"] },
        }
    return data

data = get_data(folder_path, 1)

In [27]:
tester_all_models(get_data(folder_path, 1), models)



 ++++++++++ soundness ++++++++++

 ++++++++++ iclr ++++++++++
Accuracy : 0.7368421052631579
Precision : 0.6551724137931034
Recall : 1.0
F1-score : 0.7916666666666666

 ++++++++++ neur ++++++++++
Accuracy : 0.7291666666666666
Precision : 0.6486486486486487
Recall : 1.0
F1-score : 0.7868852459016393

 ++++++++++ org ++++++++++
Accuracy : 0.975
Precision : 0.9523809523809523
Recall : 1.0
F1-score : 0.975609756097561


 ++++++++++ originality ++++++++++

 ++++++++++ iclr ++++++++++
Accuracy : 0.7861842105263158
Precision : 0.7004608294930875
Recall : 1.0
F1-score : 0.8238482384823849

 ++++++++++ neur ++++++++++
Accuracy : 0.7638888888888888
Precision : 0.6792452830188679
Recall : 1.0
F1-score : 0.8089887640449438

 ++++++++++ org ++++++++++
Accuracy : 0.95
Precision : 1.0
Recall : 0.9
F1-score : 0.9473684210526315


 ++++++++++ motivation ++++++++++

 ++++++++++ iclr ++++++++++
Accuracy : 0.8256578947368421
Precision : 0.7414634146341463
Recall : 1.0
F1-score : 0.8515406162464986

 ++++

In [28]:
tester_all_models(get_data(folder_path, 2), models)



 ++++++++++ soundness ++++++++++

 ++++++++++ iclr ++++++++++
Accuracy : 0.7006578947368421
Precision : 0.6276150627615062
Recall : 0.9868421052631579
F1-score : 0.7672634271099744

 ++++++++++ neur ++++++++++
Accuracy : 0.6736111111111112
Precision : 0.6050420168067226
Recall : 1.0
F1-score : 0.7539267015706806

 ++++++++++ org ++++++++++
Accuracy : 0.975
Precision : 0.9523809523809523
Recall : 1.0
F1-score : 0.975609756097561


 ++++++++++ originality ++++++++++

 ++++++++++ iclr ++++++++++
Accuracy : 0.7467105263157895
Precision : 0.669683257918552
Recall : 0.9736842105263158
F1-score : 0.7935656836461126

 ++++++++++ neur ++++++++++
Accuracy : 0.7465277777777778
Precision : 0.6651162790697674
Recall : 0.9930555555555556
F1-score : 0.7966573816155988

 ++++++++++ org ++++++++++
Accuracy : 0.875
Precision : 1.0
Recall : 0.75
F1-score : 0.8571428571428571


 ++++++++++ motivation ++++++++++

 ++++++++++ iclr ++++++++++
Accuracy : 0.7730263157894737
Precision : 0.6948356807511737
Rec